In [1]:
import pinecone      

pinecone.init(      
	api_key='def560e2-d944-4462-951b-8f898f867558',      
	environment='us-east1-gcp'      
)      
index = pinecone.Index('legalease')

/Users/aneeshaparajit/miniconda3/lib/python3.11/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
import pandas as pd

In [3]:
import json

data = json.load(open('../data/ner/annotatedCentralActs/188707.json'))

In [4]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

In [5]:
enc = model.encode(['THE FATAL ACCIDENTS ACT, 1855', 'How do we deal with car accidents?'])
import numpy as np

In [6]:
np.dot(enc[0], enc[1]) / (np.linalg.norm(enc[0]) * np.linalg.norm(enc[1]))

0.44789228

In [ ]:
from tqdm.autonotebook import tqdm
import os
idx = 0

for ix, file in enumerate(tqdm(os.listdir('../data/ner/annotatedCentralActs/'))):
    data = json.load(open(f'../data/ner/annotatedCentralActs/{file}'))
    # print(data.keys())
    # DEFINITIONS
    print(f'--------- Act: {data["Act Title"]}')
    try:
        definitions = {
            'Act Title': data['Act Title'],
            'Act ID': data['Act ID'],
            'text': '\n'.join([v for k,v in data['Act Definition'].items()])
        }
    except: 
        definitions = None
        
    # FOOTNOTES
    tmps = []
    for k,v in data['Footnotes'].items():
        tmps.extend([x for y, x in v.items()])
    
    foot_notes = {
        'Act Title': data['Act Title'],
        'Act ID': data['Act ID'],
        'text': '\n'.join(tmps)
    }
    
    ### PARTS
    parts = []
    # print(data.keys())
    try:
        for _, v in data['Parts'].items():
            if 'ID' in v.keys():
                sections = v['Sections']
                # __import__('pprint').pprint(sections)
                # data['']
                try:
                    for n, p in sections.items():
                        # print(p)
                        t = '\n'.join([v for k,v in p['paragraphs'].items()])
                        para = [f"{p['heading']}\n{t}"]
                        # print([p['heading']])
                        para = [p['heading']] + para
                        parts.append({'section': data['Act Title'] + f' - {n}', 'text': '\n'.join(para)})
                except:
                    pass
    except:
        try:
            for n, p in sections.items():
                # print(p)
                t = '\n'.join([v for k,v in p['paragraphs'].items()])
                para = [f"{p['heading']}\n{t}"]
                # print([p['heading']])
                para = [p['heading']] + para
                parts.append({'section': data['Act Title'] + f' - {n}', 'text': '\n'.join(para)})
        except:
            pass

    if definitions is not None:
        chunks = []
        for i in range(0, len(definitions['text']), 1000):
            mx = min(len(definitions['text']), i+1000)
            chunks.append(definitions['text'][i:mx])
        
        for chunk in tqdm(chunks):
            index.upsert(vectors=[(str(idx), model.encode(chunk).tolist(), {'Act Title': data['Act Title'], 'Act ID': data['Act ID'], 'text': chunk})])
            idx += 1

    if foot_notes is not None:
        chunks = []
        for i in range(0, len(foot_notes['text']), 1000):
            mx = min(len(foot_notes['text']), i+1000)
            chunks.append(foot_notes['text'][i:mx])
        
        for chunk in tqdm(chunks):
            index.upsert(vectors=[(str(idx), model.encode(chunk).tolist(), {'Act Title': data['Act Title'], 'Act ID': data['Act ID'], 'text': chunk})])
            idx += 1

    for part in parts:

        chunks = []

        for i in range(0, len(part['text']), 1000):
            mx = min(len(part['text']), i+1000)
            chunks.append(part['text'][i:mx])

        for chunk in tqdm(chunks):
            index.upsert(vectors=[(str(idx), model.encode(chunk).tolist(), part)])
            idx += 1

  0%|          | 0/858 [00:00<?, ?it/s]

--------- Act: THE BUILDING AND OTHER CONSTRUCTION WORKERS’ WELFARE CESS ACT, 1996


  0%|          | 0/1 [00:00<?, ?it/s]

0it [00:00, ?it/s]

--------- Act: THE INCOME-TAX ACT, 1961


...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]